<a href="https://colab.research.google.com/github/conghcmus/Fake_News_Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Khai báo thư viện

In [1]:
!git clone https://github.com/conghcmus/Fake_News_Detection.git

Cloning into 'Fake_News_Detection'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 1), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (9/9), 20.17 KiB | 10.08 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
%cd Fake_News_Detection

/content/Fake_News_Detection


In [8]:
!git

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# bert_preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

## ISOT Fake News Dataset
Link: https://onlineacademiccommunity.uvic.ca/isot/2022/11/27/fake-news-detection-datasets/

### Đọc dữ liệu

In [ ]:
True_data = pd.read_csv("datasets/ISOT Fake News Dataset/True.csv")
Fake_data = pd.read_csv("datasets/ISOT Fake News Dataset/Fake.csv")
True_data['label'] = 0
Fake_data['label'] = 1
print(True_data.shape)
print(Fake_data.shape)

(21417, 5)
(23481, 5)


In [ ]:
df_concat = pd.concat([True_data, Fake_data], axis=0)
df_concat.shape

(44898, 5)

### Text processing

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
sw = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\intern.thccong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Defining contractions
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [ ]:
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [ ]:
# Function remove_tags to handles HTML tags
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''
    return TAG_RE.sub('', text)

In [ ]:
def cleaned_text(df):
    # convert lower
    df['cleaned_text'] = df['text'].str.lower()
    # Remove html tags
    df['cleaned_text'] = df['cleaned_text'].apply(remove_tags)
    # remove punctuations
    df['cleaned_text'] = df['cleaned_text'].str.replace('[^\w\s]', '', regex=True)
    # remove numeric characters
    df['cleaned_text'] = df['cleaned_text'].str.replace('\d', '', regex=True)
    # remove contractions
    df['cleaned_text'] = df['cleaned_text'].apply(remove_contractions)
    # remove stopwords
    df['cleaned_text'] = df['cleaned_text'].apply(lambda row: " ".join(x for x in row.split() if x not in sw))
    # remove new line
    df['cleaned_text'] = df['cleaned_text'].apply(lambda row: row.replace('\n', ''))

    return df

df_concat = cleaned_text(df_concat)
df_concat[['text', 'cleaned_text']].iloc[0]

text            WASHINGTON (Reuters) - The head of a conservat...
cleaned_text    washington reuters head conservative republica...
Name: 0, dtype: object

In [ ]:
df_concat = df_concat.sample(frac = 1)
df_concat.head(10)

,title,text,subject,date,label,cleaned_text
21166,The rocky history of NAFTA,MEXICO CITY (Reuters) - Negotiators from Canad...,worldnews,"September 1, 2017",0,mexico city reuters negotiators canada mexico ...
6421,Mitt Romney Thinks Trump’s Tax Returns Will S...,"In a Facebook post on Wednesday, Mitt Romney d...",News,"May 11, 2016",1,facebook post wednesday mitt romney discussed ...
17057,Global Warming Scammer Makes Millions BUT Is J...,Global warming has been shoved down the throat...,Government News,"Oct 5, 2015",1,global warming shoved throats every american s...
22683,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,US_News,"January 22, 2016",1,st century wire says still refuse entertain mi...
17011,China official says Lincoln would have approve...,BEIJING (Reuters) - Late U.S. President Abraha...,worldnews,"October 19, 2017",0,beijing reuters late us president abraham linc...
9205,Michigan lawmakers approve Detroit school resc...,(Reuters) - Legislation approved by Michigan l...,politicsNews,"June 9, 2016",0,reuters legislation approved michigan lawmaker...
17901,North Korean missiles will be able to reach U....,MOSCOW (Reuters) - North Korea s leadership ha...,worldnews,"October 10, 2017",0,moscow reuters north korea leadership told rus...
11275,Syrian rebels say discussing evacuation from t...,BEIRUT (Reuters) - Syrian rebels in a pocket o...,worldnews,"December 27, 2017",0,beirut reuters syrian rebels pocket land near ...
20909,Hong Kong's vanishing archives and the battle ...,HONG KONG (Reuters) - For anyone digging into ...,worldnews,"September 6, 2017",0,hong kong reuters anyone digging hong kong his...
8960,Trevor Noah Compares Texans Excited About Ope...,Trevor Noah compared the excitement of the peo...,News,"January 5, 2016",1,trevor noah compared excitement people texas n...


### Build vocabulary

In [ ]:
vocab_size = 10000
max_length = 150
embedding_dim = 300

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df_concat['cleaned_text'])

In [ ]:
tokenizer.index_word

{1: '<OOV>',
 2: 'said',
 3: 'trump',
 4: 'us',
 5: 'would',
 6: 'president',
 7: 'people',
 8: 'one',
 9: 'state',
 10: 'also',
 11: 'new',
 12: 'reuters',
 13: 'donald',
 14: 'states',
 15: 'house',
 16: 'government',
 17: 'clinton',
 18: 'obama',
 19: 'republican',
 20: 'could',
 21: 'united',
 22: 'told',
 23: 'white',
 24: 'like',
 25: 'campaign',
 26: 'last',
 27: 'news',
 28: 'two',
 29: 'time',
 30: 'election',
 31: 'party',
 32: 'first',
 33: 'year',
 34: 'former',
 35: 'even',
 36: 'washington',
 37: 'country',
 38: 'years',
 39: 'many',
 40: 'hillary',
 41: 'security',
 42: 'may',
 43: 'media',
 44: 'political',
 45: 'national',
 46: 'say',
 47: 'made',
 48: 'get',
 49: 'law',
 50: 'police',
 51: 'make',
 52: 'since',
 53: 'american',
 54: 'percent',
 55: 'going',
 56: 'court',
 57: 'republicans',
 58: 'presidential',
 59: 'back',
 60: 'democratic',
 61: 'support',
 62: 'administration',
 63: 'bill',
 64: 'week',
 65: 'including',
 66: 'america',
 67: 'senate',
 68: 'russia'

### Chia tập train và test

In [ ]:
train_sentences, test_sentences, train_label, test_label = train_test_split(
    df_concat['cleaned_text'], df_concat['label'], test_size=0.2, stratify=df_concat['label']
    )

In [ ]:
train_labels = np.array(train_label)
test_label = np.array(test_label)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
train_sequences[0]

[3135,
 12,
 950,
 1,
 1121,
 8058,
 1,
 3135,
 56,
 1336,
 47,
 859,
 108,
 6270,
 1110,
 277,
 2550,
 135,
 1242,
 344,
 3054,
 404,
 1,
 3404,
 2915,
 2540,
 2541,
 1,
 1,
 1,
 108,
 1791,
 220,
 2836,
 1959,
 1,
 1,
 947,
 2199,
 1495,
 16,
 242,
 1336,
 937,
 1,
 1435,
 404,
 1,
 135,
 2073,
 45,
 1,
 798,
 7511,
 16,
 376,
 3011,
 1581,
 1073,
 565,
 35,
 497,
 1,
 8061,
 4452,
 64,
 79,
 3986,
 1,
 9100,
 425,
 982,
 937,
 281,
 38,
 398,
 1,
 1296,
 944,
 1,
 258,
 1,
 134,
 334,
 108,
 6270,
 1110,
 2550,
 627,
 332,
 1073,
 328,
 477,
 479,
 417,
 2101,
 49,
 2138,
 3461,
 623,
 5831,
 8654,
 6468,
 6270,
 2261,
 1308,
 172,
 1,
 4910,
 2006,
 2617,
 1,
 1,
 978,
 1,
 2,
 86,
 511,
 1171,
 4703,
 665,
 24,
 1664,
 1311,
 950,
 312,
 3424,
 1,
 133,
 1743,
 634,
 623,
 270,
 2310,
 404,
 1,
 982,
 1,
 1,
 672,
 640,
 6270,
 477,
 1864,
 7720,
 3732,
 404,
 332,
 1,
 2015,
 99,
 974,
 406,
 2349,
 79,
 9368,
 211,
 43,
 8755,
 260,
 640,
 3174,
 7362,
 1554,
 78,
 511,
 1341,
 

In [ ]:
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating='post', padding='post')
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating='post', padding='post')

In [ ]:
padded_train_sequences

array([[3135,   12,  950, ...,  332,    1, 2015],
       [  11,  124,   12, ...,    0,    0,    0],
       [ 868,  211,  282, ..., 5599,  115,  977],
       ...,
       [  12, 2274, 2102, ...,    0,    0,    0],
       [5153,  192,    1, ...,  304, 4079,  334],
       [3671,   12,    6, ..., 1678, 6939, 1511]])

In [ ]:
padded_train_sequences.shape

(35918, 150)

### Build model

#### TF-IDF + Linear SVM

In [ ]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers using tfidf
    ("SVC", SVC()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('SVC', SVC())])

In [ ]:
# Evaluate baseline model
base_line_score = model_0.score(test_sentences, test_label)
print(f"accuracy of baseline model: {base_line_score*100:.2f}")

accuracy of baseline model: 99.54


In [ ]:
test_sentences

19773    last night third debate fox news chris wallace...
11508    pelosi claims meeting russian ambassadorpelosi...
13071    brussels reuters theresa may must deliver offe...
16833    ankara reuters turkey bddk banking regulator u...
11794                   httpswwwyoutubecomwatchvgqxwfteyas
                               ...                        
7475     budapest reuters hungarian prime minister vikt...
21296    berlin reuters hardright alternative germany a...
18308    london reuters british police alerted suspect ...
14144    harare reuters zimbabwe former vice president ...
19408    juba reuters residents south sudan capital say...
Name: cleaned_text, Length: 8980, dtype: object

#### TF-IDF + Decision tree

In [ ]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("decision tree", DecisionTreeClassifier())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_label)

#### fastText + CNN

In [ ]:
model = Sequential()

In [ ]:
model.add(layers.Input(shape=(None, 1000)))
model.add()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

## Fake news dataset
**Link:** https://www.kaggle.com/competitions/fake-news/overview <br>
**Dataset Description** <br>

**train.csv:** A full training dataset with the following attributes:

- **id:** unique id for a news article
- **title:** the title of a news article
- **author:** author of the news article
- **text:** the text of the article; could be incomplete
- **label:** a label that marks the article as potentially unreliable
    - 1: unreliable
    - 0: reliable
    
**test.csv:** A testing training dataset with all the same attributes at train.csv without the label.

In [ ]:
Fake_news_data = pd.read_csv("datasets/Fake news dataset/train.csv")
print(Fake_news_data.shape)
Fake_news_data.head(10)

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


## Fake or real news dataset
This is the repo hosting the data for my fake vs real news project. The data is in a zipped csv file and contains almost 11000 articles tagged as either real or fake.

In [ ]:
Fake_or_real_news_data = pd.read_csv("datasets/Fake or real news dataset/fake_or_real_news.csv")
print(Fake_or_real_news_data.shape)
Fake_or_real_news_data.head(10)

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


## Fake news detection dataset

In [ ]:
Fake_news_detection_data = pd.read_csv("datasets/Fake news detection dataset/data.csv")
print(Fake_news_detection_data.shape)
Fake_news_detection_data.head(10)

(4009, 4)


,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1
5,http://beforeitsnews.com/sports/2017/09/jetnat...,JetNation FanDuel League; Week 4,JetNation FanDuel League; Week 4\n% of readers...,0
6,https://www.nytimes.com/2017/10/10/us/politics...,Kansas Tried a Tax Plan Similar to Trump’s. It...,"In 2012, Kansas lawmakers, led by Gov. Sam Bro...",1
7,https://www.reuters.com/article/us-india-cenba...,"India RBI chief: growth important, but not at ...",The Reserve Bank of India (RBI) Governor Urjit...,1
8,https://www.reuters.com/article/us-climatechan...,EPA chief to sign rule on Clean Power Plan exi...,"Scott Pruitt, Administrator of the U.S. Enviro...",1
9,https://www.reuters.com/article/us-air-berlin-...,Talks on sale of Air Berlin planes to easyJet ...,FILE PHOTO - An Air Berlin sign is seen at an ...,1
